# Section 5. Raster Data

#### Instructor: Pierre Biscaye

The content of this notebook draws on material from UC Berkeley's Spatial Data Analysis [course](https://docs.google.com/document/d/1oC10pjyeBQTenQazCpaB8Lx1b5PC1SR3WFiPgCtXqcs/edit?tab=t.0) notes by [Jaecheol Lee](https://sites.google.com/view/jaecheollee).
    
### Learning Objectives 
    
* Learn how to load and view raster data
* Understand masking of spatial objects
* Learn how to work with multidimensional raster data
* Practice basic map algebra
* Focus on visualizing spatial data
  
### Sections
1. Loading and mapping rasters using rasterio
2. Masking spatial data
3. Working with multidimensional rasters using xarray

## 0. Loading modules

In this notebook we will be working with a new library `rasterio` that helps work with raster spatial data. 

In [ ]:
# Loading modules
import pickle
import numpy as np
from scipy import stats
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import rasterio.transform
import rasterio.mask
import rasterio.warp
import rasterio.windows

from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import (Point, LinearRing,
                              Polygon, MultiPolygon)

%matplotlib inline

# 1. Loading and mapping rasters

#### What is a raster?

**Raster files** are images built from pixels — tiny color squares that, in great quantity, can form highly detailed images such as photographs. The more pixels an image has, the higher quality it will be, and vice versa.

Raster files can include many different extensions but all include the same basic type of information.

When a raster is a spatial object we can map the locations of the pixels in the image to locations on a map, and use the information in each pixel (which may have several layers) to plot spatial information.

<img src="Image/raster_vector.jpg" width = "500">

[Source: tellyourtale.com](https://tellyourtale.com/graphic-design/which-graphic-file-format-is-best-vector-and-raster-images/)

<img src="Image/smile.png" width = "350">

[Source: Wikipedia](https://en.wikipedia.org/wiki/Raster_graphics)

#### TIFF and GeoTIFF files

Two of the most common types of raster files are:
* TIFF: Tagged Image File Format for storing raster graphics images.
* GeoTIFF: TIFF file with georeferencing information embedded.

Think of a GeoTIFF file as a collection of actual rasters (we will later deal with them as `numpy.ndarray`) and a collection of metadata that contains georeferencing information. The potential metadata information includes map projection, coordinate systems, and everything else necessary to establish the exact spatial reference for the file. 

In this notebook we will be using data fromm the [Gridded Population of the World data set](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-count-rev11) produced by a team at Columbia University. We'll be using the version provided at a 15 arcminute or a 0.25 degree resolution (about 30km). This version of the dataset describes the total number of individuals that are estimated to have lived in each grid cell in the year 2000.



#### Using `rasterio` to view raster metadata.

We now start exploring how to use `rasterio` to read existing GeoTIFF files and perform some useful operations on them. 

In [ ]:
dataset = rasterio.open('Data/GPW.tif')

Let's check the metadata (data that provides information about other data) of the GeoTIFF file first.

We want to know how many layers (bands) there are in the file. A band is like a layer, or a channel. An RGB image, for example, has three channels/bands/layers. Each band shares the same georeferencing information with each other.

In [ ]:
dataset.count

Or alternatively,

In [ ]:
dataset.indexes

If you see `(1,)` that means that there is only one band in the raster. `rasterio` starts counting from 1.

We can view the coordinate reference system (CRS) by

In [ ]:
dataset.crs

The output is EPSG 4326/WGS84: World Geodetic System 1984, commonly used in GPS. This maps a point on earth to a set of longitude, latitude coordinate (notice that epsg 4326 uses the order [lon, lat]). 

Next, we can check how many grid cells there are by

In [ ]:
# Notice the sequence of height and width here!
# This is a rasterio convention, which can be different from other packages
dataset.shape

In [ ]:
dataset.height, dataset.width

See the bounds of the GeoTIFF: the coordinates of the pixels on the corners.

In [ ]:
dataset.bounds

In [ ]:
# you can of course also view all the metadata
dataset.meta

`'nodata'` refers to a special value that rasterio uses to store NaN values. `'transform'` refers to six values that rasterio uses to record the scale and the position of the raster. This can be used to convert x, y coordinate (coord in the specified crs, e.g., in lon, lat) to i, j coordinate (row and col number on a raster, coord in image space).

In [ ]:
# This converts x, y to row, col
dataset.index(0, 0)

In [ ]:
# This converts row, col to x, y
dataset.xy(0, 0)

<img src = "Image/index_xy.png" width = 500>

In [ ]:
# What are the row and column indices of a particular coordinate?
dataset.index(-120, 50)

In [ ]:
# What are the coordinates of particular indices?
print(dataset.xy(279, 120))
print(dataset.xy(279, 119))
print(dataset.xy(280, 120))

When you `rasterio.open()` something, you are not actually reading the whole raster file. You are only reading the metadata. This is a really nice property because sometimes a geotiff file could be too large to fit into your memory, and rasterio lets you read only parts of that image.

### Raster data as numpy arrays

Enough with the metadata, now we want to actually read the real data. For this we use the rasterio `read()` method, and specify what dimension or band we want to load.

In [ ]:
# Read the first dimension of the array (there's only one)
band = dataset.read(1)

This is now a numpy array that you can operate on. You can do any kind of map algebra as long as it doesn't change the georeferencing.

In [ ]:
# How dows it look?
band

In [ ]:
# Define your color palette to use in your heat map
nodes = [0, 0.5, 1]  # positions for each color from 0-1: 0 to vmin, 1 to vmax
color_scheme = ['white', 'yellow', 'red']  # corresponds to nodes
custom_cmap = LinearSegmentedColormap.from_list(
    'custom_name', list(zip(nodes, color_scheme)))
custom_cmap.set_under('gray')  # set values under vmin to gray
custom_cmap.set_over('red')  # set values over vmax to red

Now let's **plot the raster band array** using `imshow` from matplotlib.

In [ ]:
# Set up a figure environment
fig, ax = plt.subplots(figsize=(10, 4))

# Add a layer of showing the heat map
ax.imshow(band, # Data
          cmap = custom_cmap, # Color information
          extent = (-180, 180, -90, 90), # x, y scale
          vmin = 0, vmax = 50000) # set thresholds for extreme values
plt.show()

To add a **colorbar**, assign the ax.imshow to a data object.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

# Assigning ax.imshow() to data object im
# imshow method automatically add the heat map layer
im = ax.imshow(band, # Data
          cmap = custom_cmap, # Color information
          extent = (-180, 180, -90, 90), # x, y scale
          vmin = 0, vmax = 100000)
fig.colorbar(im)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Population of the World (Source: CIESIN 2018)')
plt.show()

In [ ]:
# Zooming in by using set_xlim and set_ylim methods
fig, ax = plt.subplots(figsize=(10, 4))

im = ax.imshow(band, # Data
          cmap = custom_cmap, # Color information
          extent = (-180, 180, -90, 90), # x, y scale
          vmin = 0, vmax = 100000)

fig.colorbar(im)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Population of the World: Europe')

ax.set_xlim((-20, 50))
ax.set_ylim((30, 70))

plt.show()

In [ ]:
# Change the colors? 

nodes = [0, 0.33, 0.66, 1]  # positions for each color from 0-1
color_scheme = ['bisque', 'yellow', 'olivedrab', 'navy']  # corresponds to nodes
custom_cmap = LinearSegmentedColormap.from_list(
    'custom_name', list(zip(nodes, color_scheme)))
custom_cmap.set_under('gray')  # set values under vmin to gray
custom_cmap.set_over('purple')  # set values over vmax to black

fig, ax = plt.subplots(figsize=(10, 4))

im = ax.imshow(band, # Data
          cmap = custom_cmap, # Color information
          extent = (-180, 180, -90, 90), # x, y scale
          vmin = 0, vmax = 100000)

fig.colorbar(im)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Population of the World: Europe')

ax.set_xlim((-20, 50))
ax.set_ylim((30, 70))

plt.show()

You can change colors as you want.

<img src="Image/colors.png" width = "500">
[Link](https://matplotlib.org/3.3.2/gallery/color/named_colors.html)

### Practicing map algebra

Let's practice doing some map algebra by mapping log population density over the world.

To do this, we will need
* To figure out how large each grid cell is;
* To assume that 1 degree of latitude is 111.11 km and that 1 degree of longitude at the equator is 111.11 km;
* To create a lat-lon grid so that you can compute an array that is the same size as the original data set but where each element describes the area of the grid cell;
* To use map algebra to convert $population$ to $population/area$ to $\log_{10}(population/area)$, i.e. so a value of 1 indicates a density of 10 people per sq. km, 2 indicates a density of 100 people per sq. km, etc.

First, let's make the grid. The resolution of the image is 15 arcminutes, or .25 degrees.
The center of each pixel is therefore located $\delta/2$ from the edge, so let's define two arrays representing the latitude and longitude of each pixel center (starting $\delta/2$ from the edges of the image, which are -180 and 180 for longitude, and 90 and -90 for latitude).
Then, we make a meshgrid of these points.

In [ ]:
delta = .25
lons = np.arange(start=-179.875, stop=179.875 + delta, step=delta)
# note the reversed direction of the step below, because we are going from north to south
lats = np.arange(start=89.875, stop=-89.875 - delta, step=-delta)
lon_grid, lat_grid = np.meshgrid(lons, lats)

When we examined the numpy array `band` earlier, we saw many values of -3.402823e+38. This is how "negative infinity" is stored, and represents grid cell containing no individuals. We will have to change this before calculating population density.

Now, on to the calculation!

In [ ]:
# Latitude (y): 1 deg = 111.11 km
# Longitude (x): 1 deg = 111.11 * cos(latitude in radian) km
# Cell size is 0.25 degrees
# Area = Ykm * Xkm
area = (111.11 * .25) * (111.11 * .25 * np.cos(lat_grid / 180 * np.pi))
# handle negative values
band[band < 1e-1] = 1e-1
log10_density = np.log10(band / area)

In [ ]:
# Map it!
fig, ax = plt.subplots(figsize=(10, 4))
im = ax.imshow(log10_density, cmap=custom_cmap,
               extent=(-180, 180, -90, 90),
               vmin=0, vmax=3)
fig.colorbar(im)
# label axes and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('log10(Population Density) (Source: CIESIN 2018)')
plt.show()

# 2. Masking

A useful operation with spatial data is masking. Masking allows you to hide or 'mask' or clip specific areas, to make a cleaner figure.

Suppose we were interested in the population of Mexico, and wanted to only show population data for Mexico. 

We will first load a shapefile with the boundaries of Mexico, which we can then use as a mask for the population raster.

In [ ]:
# Load Mexico shp file
mex = gpd.read_file('Data/MEX_adm0.shp')
mex

How can we erase the areas which are out of Mexico? With the `rasterio.mask.mask()` function, we can clip the raster with the shapefile.

In [ ]:
clipped_array, _ = rasterio.mask.mask(dataset, mex.geometry, nodata = -1)
print(clipped_array.shape)

In [ ]:
# Remove single-dimensional entries from the shape of an array.
# using np.squeeze
clipped_array = clipped_array.squeeze(axis = 0)
print(clipped_array.shape)
clipped_array

In [ ]:
# Figuring out the right coordinates to zoom on Mexico
mex.geometry[0].bounds

In [ ]:
# Plotting starts
fig, ax = plt.subplots(figsize=(10, 4))
im = ax.imshow(clipped_array, cmap=custom_cmap,
               extent=(-180, 180, -90, 90),
               vmin=0, vmax=100000)
fig.colorbar(im)

# Zoom in on Mexico using it coordinates
ax.set_xlim((mex.geometry[0].bounds[0]-1, mex.geometry[0].bounds[2]+1))
ax.set_ylim((mex.geometry[0].bounds[1]-1, mex.geometry[0].bounds[3]+1))

# Label axes and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Population in Mexico (Source: CIESIN 2018)')

# Extra
ax.fill(-110,15,color='bisque', label='Unpopulated')
ax.legend(loc=3, fontsize=10)
plt.show()

### Affine transformations and masking

Suppose we want to mask particular polygons. Let's work with a map for administrative units in France - regions.

We can load and map these boundaries easily using `geopandas`.

In [ ]:
df = gpd.read_file('Data/gadm41_FRA_1.shp') 

fig, ax = plt.subplots(figsize=(7, 7))
df.plot(ax=ax, color='white', edgecolor='black')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('France')
plt.show()

How can we create a mask for one administrative unit?
Previously, we created a mask using raterio.mask.mask:

```
band = rasterio.open('Data/GPW.tif').read(1)
mex = gpd.read_file('Data/MEX_adm0.shp')
clipped_array, _ = rasterio.mask.mask(dataset, mex.geometry, nodata = -1).squeeze(axis=0)

```

Now we do not have a `rasterio` file at the moment. We need to create a raster file first. 
More precisely, we need to convert the shape file of France regions (`df`) into a raster data. 
For doing this, we first need to distort the CRS file a bit. 
We need to convert the shape file with the longitude and the latitude into a raster file with many pixels. 

What we will utilize is `Affine` that divides the shape file by numerous pieces and fits it into a rectangular box.

In [ ]:
from rasterio.features import geometry_mask
from affine import Affine

In [ ]:
help(Affine) # Focus on the 2-D transformations

Fortunately, we do not need to memorize all the transformation types presented above. ```Affine``` function provides a more intuitive way of doing affine transformations. We focus on **the destination of the transformation**, not the departure. 

#### Introducing affine transformations to georeference rasters

Affine(a, b, c, d, e, f)
- a = width 'resolution' of a pixel
- b = row rotation (typically zero) # 
- c = x-coordinate of the upper-left corner of the upper-left pixel
- d = column rotation (typically zero)
- e = height 'resolution' of a pixel (typically negative)
- f = y-coordinate of the of the upper-left corner of the upper-left pixel
##### see more in https://docs.geotools.org/latest/userguide/tutorial/affinetransform.html

Let's move forward supposing we want pixels at a 0.1 degree resolution.

In [ ]:
# An easy way to get the extent of the whole area
df['new_column'] = 0
df_new = df.dissolve(by='new_column')
df_new.bounds

In [ ]:
xmin, ymin, xmax, ymax = -5.2, 41.2, 9.6, 51.2
res = 0.1

Now, we have a set of information regarding the 'destination' of our shape file. We further need the `geometry_mask` function.

In [ ]:
help(geometry_mask)

In [ ]:
masks = geometry_mask(
        geometries = [df['geometry'][0]], # Geometry of the shape you want to analyze
        out_shape=(int(np.round((ymax - ymin) / res)), # the number of `pixel-widths` in the y-axis
                   int(np.round((xmax - xmin) / res))), # the number of `pixel-heights` in the x-axis
        transform = Affine(res, 0, xmin, 0, -res, ymax), # Destination information
        all_touched=True, # Including all the pixels if boundaries just 'touch' them
        invert=True) # 1/0 inversion

In [ ]:
df.shape

In [ ]:
# How does it look when it is plotted?
plt.imshow(masks, extent=(xmin, xmax, ymin, ymax))
for i in range(13):
    geom = df.loc[i, 'geometry']
    # If it's a Polygon, just plot its exterior
    if geom.geom_type == 'Polygon':
        x, y = geom.exterior.xy
        plt.plot(x, y, 'k-')
    # If it's a MultiPolygon, iterate over each Polygon
    elif geom.geom_type == 'MultiPolygon':
        for poly in geom.geoms:  
            x, y = poly.exterior.xy
            plt.plot(x, y, 'k-')
plt.show()

# 3. Working with multidimensional data

The GPW raster had only one band, but often we will be working with multidimensional raster data, where each pixel/point has multiple variables/pieces of information associated with it. 

We will review using `xarray`s as a useful way for working with raster data that can accommodate such rasters.

First, we'll import the `xarray` package.

In [ ]:
import xarray as xr

### Quick Overview of `xarray`

<img src="Image/xarray.jpeg" width = "900">

`xarray` is a great tool for handling high dimensional arrays, for example meteorological/atmospheric data over time.

- What is a high dimensional array? 3-D, 4-D, 5-D...
- Why is it useful? To store dynamic fields.
- A heuristic explanation: Tonnes of rasters in one data object
    - In `rasterio`, each band has one static field. That means, one location, one number.
    - In `xarray`, one data object can contain dynamic fields. That means, one location, many numbers or a lot of sets of information.

#### Some useful links:
- [Why do we want to use `xarray`?](http://xarray.pydata.org/en/stable/why-xarray.html)
- [Core data structure](http://xarray.pydata.org/en/stable/why-xarray.html)
- Two basic sorts of functionality:
    - [Apply operations over dimensions](http://xarray.pydata.org/en/stable/computation.html)
    - [Indexing](http://xarray.pydata.org/en/stable/indexing.html)
    
#### NetCDF4?
NetCDF (network Common Data Form, or .nc) is a file format for storing multidimensional scientific data (variables) such as temperature, humidity, pressure, wind speed, and direction. 

Let's use `xarray` to open a multidimensional raster of rainfall data over time in Australia.

In [ ]:
ds = xr.open_dataset('Data/AustraliaRainfall.nc')

In [ ]:
# Data Description
ds

What does this tell us?
* Data are monthly rainfall in mm for 1970-2007.
* The dataset has 4 dimensions that uniquely identify an observation (address): lat, lon, year, and month.
* The variable defined for each observation is called `rainfall`.

How many values does each dimension take?

In [ ]:
# Dimensions (of addresses)
ds.dims

In [ ]:
# Checking the coordinates (or addresses)
# Here, 'coords' are not restricted to indicating longitude and latitude.
ds.coords

In [ ]:
# Which datasets does ds have for each set of address?
ds.data_vars

In [ ]:
# Accessing the rainfall dataset
ds['rainfall']

In [ ]:
# Accessing the rainfall values
ds['rainfall'].values

In [ ]:
# What is the shape of the rainfall values? This is a 4-dimensional object
ds['rainfall'].values.shape

In [ ]:
# The basic way of indexing. What does this mean?
ds['rainfall'].values[:, :, 1, 5]

In [ ]:
# Indexing based on sel method
# This is the same as above! 
ds['rainfall'].sel(year = 1971, month = 6).values

### Plotting multidimensional data

Now let's plot these data. To start out, we'll plot the data in 2 dimensions. That means we'll have to restrict the data we are trying to plot.

Let's restrict to a particular year and month to start.

In [ ]:
# Plotting with imshow
plt.imshow(ds['rainfall'].sel(year = 1971, month = 6), origin = 'lower')
# origin = 'upper' is the default. The value in the location (0, 0) to the left-top corner.
# origin = 'lower' puts the value in the location (0, 0) to the left-bottom corner. 
plt.show()

In [ ]:
# What's the difference between this and the figure using the imshow function?
ds['rainfall'].sel(year=1971, month=6).plot();

What if we wanted to plot **data over time**? We will need to restrict the space dimensions.

For example, let's plot monthly rainfall for 2000 in an arbitrary location in Australia, and annual rainfall in April for the same location.

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(12, 4))
ds['rainfall'].sel(year = 2000, lat = -27, lon = 135).plot(ax=ax0)
ds['rainfall'].sel(month = 4, lat = -27, lon = 135).plot(ax=ax1);

### Calculations with `xarray`

`xarray` has a variety of methods we can use to perform calculations. For example, let's calculate mean rainfall for January across years.

In [ ]:
ds['rainfall'].sel(month = 1).mean(dim = 'year').plot()

In [ ]:
# What is this plotting?
ds['rainfall'].mean(dim = 'year').mean(dim = 'month').plot()

In [ ]:
# What is this plotting?
ds['rainfall'].sum(dim = 'month').mean(dim = 'year').plot()

In [ ]:
fig, (ax1, ax4, ax8) = plt.subplots(ncols=3, figsize=(12, 3))
ds['rainfall'].sel(month=1).mean(dim='year').plot(ax=ax1, vmax=200)
ds['rainfall'].sel(month=4).mean(dim='year').plot(ax=ax4, vmax=200)
ds['rainfall'].sel(month=8).mean(dim='year').plot(ax=ax8, vmax=200)
plt.tight_layout()

You can also create new variables to plot by using **map algebra**.

For example, let's calculate how mean rainfall in January compares to mean rainfall in other months across time.

In [ ]:
# You can also create new variables to plot by using map algebra.
jan_mean = ds['rainfall'].mean(dim='year').sel(month=1)
month_mean = ds['rainfall'].mean(dim='month').mean(dim='year')
((jan_mean - month_mean) / month_mean).plot(cmap='bwr')
# Explore https://matplotlib.org/stable/users/explain/colors/colormaps.html